In [1]:
import os
import openpyxl
import locale
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
from sqlalchemy import create_engine, exc

load_dotenv()

path_dados = os.getenv('DUSNEI_DATA_DIRECTORY_RELATORIO_PENDENCIAS')

In [2]:
def get_db_engine():
    try:
        db_url = os.getenv('DUSNEI_URL')
        engine = create_engine(db_url)
        # Test connection
        with engine.connect() as connection:
            print('Conexão estabelecida!')
            pass
        print('Banco de dados conectado!')
        return engine
    except exc.SQLAlchemyError as e:
        print(f"Error: {e}")
        return None
        


In [9]:
def formas_pagamento(conn):
    formas_pagamento = []

    query = """
        SELECT
            fpgt.fpgt_codigo AS fpgt_cod,
            fpgt.fpgt_descricao AS fpgt_nome
        FROM fpgto AS fpgt
    """
    df = pd.read_sql_query(query, conn)

    # Adicionando os códigos à lista
    formas_pagamento.extend(df['fpgt_cod'].tolist())

    return formas_pagamento

In [4]:
def pendencias_clientes_query(conn, codigo_fpgt):
    tables = [
        'movprodd0123', 'movprodd0223', 'movprodd0323', 'movprodd0423', 'movprodd0523', 'movprodd0623', 
        'movprodd0723', 'movprodd0823', 'movprodd0923', 'movprodd1023', 'movprodd1123', 'movprodd1223'
    ]
    
    query = (f"""
        (
            SELECT
                mprd.mprd_dcto_codigo AS doc_cod, 
                mprd.mprd_transacao AS transacao,
                clie.clie_cnpjcpf AS cnpj_cpf,
                clie.clie_codigo AS cod_clie,
                mprd.mprd_datamvto AS data,
                mprd.mprd_numerodcto AS nfe,
                prod.prod_codbarras AS cod_barras,
                prod.prod_codigo AS cod_prod,
                (mprd.mprd_qtde * prod.prod_pesoliq) AS quantity,
                mprd.mprd_valor AS amount,
                mprc.mprc_vend_codigo AS cod_vend,
                SUBSTRING(clie.clie_cepres, 1,5) ||'-'|| SUBSTRING(clie.clie_cepres, 6,3) AS cep
            FROM {tables} AS mprd 
            LEFT JOIN movprodc AS mprc ON mprd.mprd_operacao = mprc.mprc_operacao
            LEFT JOIN produtos AS prod ON mprd.mprd_prod_codigo = prod.prod_codigo
            LEFT JOIN clientes AS clie ON mprc.mprc_codentidade = clie.clie_codigo
            WHERE mprd_status = 'N' 
            AND mprd.mprd_dcto_codigo IN ('6666','6667','6668','7339','7335','7338','7337','6680','6890','7260','7263','7262','7268','7264','7269', '7267', '7319', '7318')
        )  
    """)
    
    df = pd.read_sql_query(query, conn)
    
    filtered_dfs = [df.dropna(how='all', axis=1) for df in [df for table in tables]]
    df_tratado = pd.concat(filtered_dfs)
    
    return df_tratado

In [5]:
def gerar_excel(df, ws, codigo_vendedor):
    ws.title = f"Vendedor {codigo_vendedor}"
    
    columns_title = [
        'CodClie','RazaoSocial','Nota','Valor','Vencimento','Supervisor','Vendedor','C.Descrição'
    ]
    
    row_title = 1

    for col_num, column_title in enumerate(columns_title, 1):
        cell = ws.cell(row=row_title, column=col_num)
        cell.value = column_title

    for index, row in enumerate(df.itertuples(), start=2):
        ws.cell(row=index, column=1).value = row.entidade_cod
        ws.cell(row=index, column=2).value = row.razaosocial
        ws.cell(row=index, column=3).value = row.nota
        ws.cell(row=index, column=4).value = row.valor
        ws.cell(row=index, column=4).number_format = 'R$ #,##0.00'
        ws.cell(row=index, column=5).value = row.datavcto.strftime("%Y/%m/%d")
        ws.cell(row=index, column=6).value = row.supe_nome
        ws.cell(row=index, column=7).value = row.vend_nome
        ws.cell(row=index, column=8).value = row.descricao_contabil


In [6]:
def gerar_excel_resumo(df, wb):
    resumo_df = df.groupby('descricao_contabil', as_index=False)['valor'].sum()
      
    ws_resumo = wb.create_sheet(title="Resumo Contas")
    
    columns_title = [
        'Descrição','Valor'
    ]

    row_title = 1

    for col_num, column_title in enumerate(columns_title, 1):
        ws_resumo.cell(row=row_title, column=col_num).value = column_title

    for index, row in enumerate(resumo_df.itertuples(), start=2):
        ws_resumo.cell(row=index, column=1).value = row.descricao_contabil
        ws_resumo.cell(row=index, column=2).value = row.valor
        # Define o estilo de número da célula para ter duas casas decimais
        ws_resumo.cell(row=index, column=2).number_format = 'R$ #,##0.00'


In [7]:
def gerar_excel_cidade(df, wb):
    # Agrupar por 'vend_cod' e somar os valores
    resumo_df = df.groupby('municipio', as_index=False)['valor'].sum()

    # Criar uma nova aba no workbook
    ws_resumo = wb.create_sheet(title="Resumo Municipio")

    # Títulos das colunas
    columns_title = ['Municipio', 'Valor']
    row_title = 1

    # Preencher títulos das colunas
    for col_num, column_title in enumerate(columns_title, 1):
        ws_resumo.cell(row=row_title, column=col_num).value = column_title

    # Preencher dados nas células
    for index, row in enumerate(resumo_df.itertuples(), start=2):
        ws_resumo.cell(row=index, column=1).value = row.municipio
        # ws_resumo.cell(row=index, column=2).value = f"{row.valor:.2f}".replace(".",",")
        ws_resumo.cell(row=index, column=2).value = row.valor
        # Define o estilo de número da célula para ter duas casas decimais
        ws_resumo.cell(row=index, column=2).number_format = 'R$ #,##0.00'

In [ ]:
def teste_loop():
    conn = get_db_engine()
    lista_vendedor = vendedores(conn)

    for codigo_vendedor in lista_vendedor:
        df = pendencias_clientes_query(conn, codigo_vendedor)
        
        if df.empty:
            print(f"Não há dados para o vendedor {codigo_vendedor}. Arquivo não será gerado.")
            continue  # Pula para o próximo vendedor

    
        locale.setlocale(locale.LC_MONETARY, 'pt-BR.UTF-8')
        wb = openpyxl.Workbook()
        ws = wb.active
        
        gerar_excel(df, ws, codigo_vendedor)
        gerar_excel_resumo(df, wb)
        gerar_excel_cidade(df, wb)

        data_pasta = datetime.now().strftime("%Y-%m-%d")
        nome_arquivo = (f'{codigo_vendedor}-pendencias-{data_pasta}')
        diretorio = f'{path_dados}/{data_pasta}/vendedores'
        if not os.path.exists(diretorio):
            os.mkdir(diretorio)
        local_arquivo = os.path.join(f'{diretorio}/{nome_arquivo}.xlsx')
        wb.save(local_arquivo)
    
    print("Arquivos criados")


teste_loop()